In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
df = pd.read_csv("/kaggle/input/weather-dataset-rattle-package/weatherAUS.csv")
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
#write by xiaofeng
df.shape #
#null value count for each column
print(df.isnull().sum())
#title of columns
df_cols_tle = df.columns[df.dtypes != object]
df_cols_tle

In [ ]:
# Here i am printing the min and max values for all the columns and their types - Dave
def print_max_min_type_columns(df):
    for key in df:
        #Dropping NA values from the column
        df[key] = df[key].replace(np.nan, 0)
        # Some columns can't have max or min values
        try:
            #if it can't be converted to a float, then the max is irrelevant, and will print unique values instead
            float(max(df[key]))
            print("Maximum value for "+ key+" : " + str(max(df[key])))
            print("Min value for "+ key+" : " + str(min(df[key])))
            print("Type for " + key + " : "+ str(type(max(df[key]))))
            # Spacing added to visualy seperate the columns
            print("\n")
        # Instead I will print their unique value
        except:
            uniquevalues = df[key].unique()
            print("Unique values for " + key + ": "+ str(df[key].unique()))
            print("Type for " + key + " : " + str(type(uniquevalues[1])))
            print("\n")
#Based on the printed maximum and min

In [ ]:
#Replacing the missing data - Dave
# Thanks to the summation done of all the missing values, we can see how many columns contain missing values.
# As there are multiple ways to clean these missing values, I will create multiple functions to clean in different ways.
# Removing columns based on a percentage
def removing_columns_NApercentage(data, percent):
    data = data.loc[:, data.isnull().sum() < percent*data.shape[0]]
    return data
    
# example: df = removing_columns_NApercentage(df, 0.4)

#This print doesn't replace the nan's with zero
def print_max_min_type_columns2(df):
     for key in df:
        
        # Some columns can't have max or min values
         try:
            #if it can't be converted to a float, then the max is irrelevant, and will print unique values instead
            float(max(df[key]))
            print("Maximum value for "+ key+" : " + str(max(df[key])))
            print("Min value for "+ key+" : " + str(min(df[key])))
            print("Type for " + key + " : "+ str(type(max(df[key]))))
            # Spacing added to visualy seperate the columns
            print("\n")
         # Instead I will print their unique value
         except:
            uniquevalues = df[key].unique()
            print("Unique values for " + key + ": "+ str(df[key].unique()))
            print("Type for " + key + " : " + str(type(uniquevalues[1])))
            print("\n")

# This func is to print the percentage of na values in each column
def printing_columns_NApercentage(data):
    for key in data:
        total_nulls = data[key].isnull().sum()
        rows = data[key].shape
        print("The percentage of nulls in column: " + key+" is " + str((total_nulls/rows)*100))
        
        
#This function will remove the rows when their na percentage exceeds the percent threshold        
def removing_rows_napercentage(data, percent):
    num_cols = data.shape[1]
    thresh = (1-percent)*num_cols
    data.dropna(axis=0, thresh=thresh)
    return data


#This function will remove rows where RainToday is 0/null
def removing_RainToday_null(dataframe):
    dataframe = dataframe[dataframe.RainToday.notna()]
    return dataframe

def removing_RainTomorrow_nullrows(dataframe):
    dataframe = dataframe[dataframe.RainTomorrow.notna()]
    return dataframe

#This function converts na values to the mean or to the most frequent value
def na_to_mean(dataframe):
    for key in dataframe:
        try:
            float(max(dataframe[key]))
            dataframe[key] = dataframe[key].replace(np.nan, np.mean(dataframe[key]))
        except:
            dataframe[key] = dataframe[key].replace(np.nan, dataframe[key].value_counts().idxmax())
    return dataframe

#This forward fill with replace na values with the value that preceded it the day before
# Have also added back fill after incase there are any na values in the first row/index/day
def forward_fill(dataframe):
    return dataframe.ffill().bfill()


In [ ]:
cleaned = forward_fill(df)
cleaned.head()
#cleaning the data with forward fill

In [ ]:
from sklearn import linear_model

In [ ]:
plt.figure(figsize=(10, 10))
plt.plot(cleaned['MaxTemp'], cleaned['Rainfall'], color='red')
plt.xlabel('MaxTemp')
plt.ylabel('Rainfall')
plt.title('Relationship of max temperature on rainfall')

In [ ]:
plt.figure(figsize=(10, 10))
plt.plot(cleaned['Date'], cleaned['Rainfall'], color='red')
plt.xlabel('Date 2008-2017')
plt.ylabel('Rainfall')
plt.title('Rainfall over time')

In [ ]:
plt.figure(figsize=(10, 10))
plt.plot(cleaned['WindGustSpeed'], cleaned['Rainfall'], color='red')
plt.xlabel('WindGustSpeed')
plt.ylabel('Rainfall')
plt.title('Relationship of WindGustSpeed on rainfall')

In [ ]:
plt.figure(figsize=(10, 10))
plt.plot(cleaned['Pressure9am'], cleaned['Rainfall'], color='red')
plt.xlabel('Pressure9am')
plt.ylabel('Rainfall')
plt.title('Relationship of Pressure9am on rainfall')

In [ ]:
X = cleaned['Temp9am']
y = cleaned['Rainfall']
plt.figure(figsize=(10, 10))
plt.plot(X, y, 'o')
plt.xlabel('Temp9am')
plt.ylabel('Rainfall')
m, b = np.polyfit(X, y, 1)
plt.plot(X, m*X + b)

In [ ]:
#converting the data so yes/no values are changed to 0 and 1 
booleanRainTomorrow = cleaned

booleanRainTomorrow['RainTomorrow'] = booleanRainTomorrow['RainTomorrow'].map(
                   {'Yes':1 ,'No':0})

booleanRainTomorrow.tail()

In [ ]:
#converting the data so yes/no values are changed to 0 and 1 
booleanRainTomorrow['RainToday'] = booleanRainTomorrow['RainToday'].map(
                   {'Yes':1 ,'No':0})

booleanRainTomorrow.tail()

In [ ]:
booleanRainTomorrow.loc[(booleanRainTomorrow['RainToday'] == 1) & booleanRainTomorrow['RainTomorrow'] == 1]
#looking at the total times it rains both today and tomorrow

In [ ]:
booleanRainTomorrow.loc[(booleanRainTomorrow['RainToday'] == 1)]
#looking at how often it rains today

In [ ]:
print(15712/32762)
#using the previous outputs to calculate how often it rains tomorrow if it rains today

In [ ]:
booleanRainTomorrow.loc[(booleanRainTomorrow['RainToday'] == 0)]

In [ ]:
booleanRainTomorrow.loc[(booleanRainTomorrow['RainToday'] == 0) & booleanRainTomorrow['RainTomorrow'] == 1]

In [ ]:
print(17047 /112698 )
#using the previous outputs to calculate how often it rains tomorrow if it hasn't rained today

In [ ]:
y = cleaned['Location']
x = cleaned['Rainfall']
plt.figure(figsize=(80, 30))
plt.plot(X, y, 'o')
plt.xlabel('Temp9am')
plt.ylabel('Rainfall')
#looking at suburbs wasn't too helpfull to visualise in this way

In [ ]:
#running boxplots on different collums to see their spread
fig1, ax1 = plt.subplots()
ax1.set_title('Boxplot of humidity at 3pm')
ax1.boxplot(booleanRainTomorrow['Humidity3pm'])

In [ ]:

booleanRainTomorrow.loc[(booleanRainTomorrow['Humidity3pm'] > 90) & booleanRainTomorrow['RainTomorrow'] == 1]

In [ ]:
booleanRainTomorrow.loc[(booleanRainTomorrow['Humidity3pm'] > 90)]

In [ ]:
print(4518/5377)
#using the counts from above to calculate If it had rained today the chance it would then rain tomorrow

In [ ]:
fig1, ax1 = plt.subplots()
ax1.set_title('Boxplot of Pressure at 3pm')
ax1.boxplot(booleanRainTomorrow['Pressure3pm'])

In [ ]:
booleanRainTomorrow.loc[(booleanRainTomorrow['Pressure3pm'] > 1020) & booleanRainTomorrow['RainTomorrow'] == 1]

In [ ]:
booleanRainTomorrow.loc[(booleanRainTomorrow['Pressure3pm'] > 1020)]

In [ ]:
print(4911/35506)
#using the counts to see if pressure above certain values significantly affected rain tomorrow

In [ ]:
fig1, ax1 = plt.subplots()
ax1.set_title('Boxplot of Cloud at 3pm')
ax1.boxplot(booleanRainTomorrow['Cloud3pm'])

In [ ]:
booleanRainTomorrow.loc[(booleanRainTomorrow['Cloud3pm'] > 7) & booleanRainTomorrow['RainTomorrow'] == 1]

In [ ]:
booleanRainTomorrow.loc[(booleanRainTomorrow['Cloud3pm'] > 7)]

In [ ]:
print(11244/32829)

In [ ]:
booleanRainTomorrow.loc[(booleanRainTomorrow['Cloud3pm'] > 5) & (booleanRainTomorrow['Humidity3pm'] > 90) & booleanRainTomorrow['RainTomorrow'] == 1]
#combining both humidity and cloud to see how they can work together to determine rain tomorrow

In [ ]:
booleanRainTomorrow.loc[(booleanRainTomorrow['Cloud3pm'] > 5) & (booleanRainTomorrow['Humidity3pm'] > 90)]

In [ ]:
print(3656 /4356 )
#seeing how often it rained tomorrow if cloud was above 5 and humidity above 90% 